# A4: Do you AGREE?

This assignment will guide you in training a pre-trained model like BERT from scratch, focusing onleveraging text embeddings to capture semantic similarity. 

Additionally, we will explore how to adapt the loss function for tasks like Natural Language Inference (NLI) to enhance the model’s ability to understand semantic relationships between texts.

#### Step 0: Prepare Environment - Import Libraries and select device

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

import datasets, math, re, random, time
from collections import Counter
from tqdm import tqdm

In [2]:
# mimimum required torch version for MPS support "1.12+"
torch.__version__

'2.10.0'

In [3]:
# Hugging Face Hub login token
# Make sure to set the HF_TOKEN environment variable in your .env file with your Hugging Face token
import os
HF_TOKEN = os.environ.get("HF_TOKEN")


In [4]:
# universal device selection: use gpu if available, else cpu
import torch

def get_device():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # Clear CUDA cache to free up memory 
        return torch.device("cuda")      # NVIDIA GPU
    elif torch.backends.mps.is_available():
        torch.mps.empty_cache()  # Clear MPS cache to avoid memory issues
        return torch.device("mps")       # Apple Silicon GPU
    else:
        torch.empty_cache()  # Clear CPU cache to free up memory
        return torch.device("cpu")

device = get_device()

print(f"Using device: {device}")

Using device: mps


## Task 1. Training BERT from Scratch - Based on Masked Language Model/BERT-update.ipynb, modify as follows: (2 points)

### 1.1) Implement Bidirectional Encoder Representations from Transformers (BERT) from scratch, following the concepts learned in class.

[BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding](https://arxiv.org/pdf/1810.04805)

#### Step 1: Data Acquisition

In [5]:
# Hugging Face Hub login token
# Make sure to set the HF_TOKEN environment variable in your .env file with your Hugging Face token
from dotenv import load_dotenv
load_dotenv()

import os
HF_TOKEN = os.environ.get("HF_TOKEN")

Using dataset from [Salesforce wikitext](https://huggingface.co/datasets/Salesforce/wikitext)

Info about Dataset:
wikitext-103-raw-v1
```sh
    Size of downloaded dataset files: 191.98 MB
    Size of the generated dataset: 549.42 MB
    Total amount of disk used: 741.41 MB
```

In [6]:
import os
from datasets import load_dataset

# data folder are not uploaded to Github.
_DATA_PATH = "../data/wikitext-103"
_DATA_FILENAME = os.path.join(_DATA_PATH, "wikitext-103-train.arrow")
os.makedirs(_DATA_PATH, exist_ok=True)

if not os.path.exists(_DATA_FILENAME):
    # Download and save to local folder
    dataset_train = load_dataset("wikitext", "wikitext-103-raw-v1", split="train", cache_dir=_DATA_PATH)
    dataset_valid = load_dataset("wikitext", "wikitext-103-raw-v1", split="validation", cache_dir=_DATA_PATH)
    dataset_test = load_dataset("wikitext", "wikitext-103-raw-v1", split="test", cache_dir=_DATA_PATH)

else:
    # Load from local Parquet file
    from datasets import Dataset
    dataset_train = Dataset.from_parquet(_DATA_FILENAME)
    dataset_valid = Dataset.from_parquet(os.path.join(_DATA_PATH, "wikitext-103-validation.arrow"))
    dataset_test = Dataset.from_parquet(os.path.join(_DATA_PATH, "wikitext-103-test.arrow"))
    print("Loaded datasets from local Parquet files.")

print(f"Dataset size: {len(dataset_train)}")
print(f"Validation set size: {len(dataset_valid)}")
print(f"Test set size: {len(dataset_test)}")

Dataset size: 1801350
Validation set size: 3760
Test set size: 4358


Check dataset 

In [7]:
dataset_train

Dataset({
    features: ['text'],
    num_rows: 1801350
})

In [8]:
dataset_test

Dataset({
    features: ['text'],
    num_rows: 4358
})

In [9]:
dataset_valid

Dataset({
    features: ['text'],
    num_rows: 3760
})

In [10]:
# Display the first 5 entries in the dataset 
# with only the first 80 characters of the text for brevity
[text[:80] for text in dataset_train[:5]['text']]

['',
 ' = Valkyria Chronicles III = \n',
 '',
 ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Va',
 ' The game began development in 2010 , carrying over a large portion of the work ']

#### Step 2: Data Preprocessing

In [11]:
sentences = [re.sub("[.,!?\\-]", '', t.lower()) for t in dataset_train["text"] if t.strip()]

#### Step 3: Tokenization 

using spaCy tokenizer or NLTK

In [12]:
raw_text = "".join(dataset_train['text'])
print(f"Total characters in raw text: {len(raw_text)}")


Total characters in raw text: 538294333


Too big dataset. It will fail spacy max_lenght limit validation. The validation exists to prevent memory allocation error. Don't concatenate, but use for loop on whole dataset

In [13]:
dataset_train["text"][1]

' = Valkyria Chronicles III = \n'

Since spaCy is slow, Using NLTK.

The command nltk.download('punkt') downloads the "punkt" tokenizer models for NLTK. "Punkt" is a pre-trained model used by NLTK for sentence splitting and word tokenization in English and other languages. Without downloading "punkt"

In [14]:
import nltk

_DOWNLOAD_DIR = "../models/nltk_data"
os.makedirs(_DOWNLOAD_DIR, exist_ok=True)
nltk.download('punkt', download_dir=_DOWNLOAD_DIR)
nltk.download('punkt_tab', download_dir=_DOWNLOAD_DIR)
from nltk.tokenize import word_tokenize


nltk.data.path.append(_DOWNLOAD_DIR)
tokenized_texts = [word_tokenize(text) for text in sentences]

# Example: print the first 5 tokenized samples
for tokens in tokenized_texts[:5]:
    print(tokens)

[nltk_data] Downloading package punkt to ../models/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to ../models/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


['=', 'valkyria', 'chronicles', 'iii', '=']
['senjō', 'no', 'valkyria', '3', ':', 'unrecorded', 'chronicles', '(', 'japanese', ':', '戦場のヴァルキュリア3', 'lit', 'valkyria', 'of', 'the', 'battlefield', '3', ')', 'commonly', 'referred', 'to', 'as', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', 'is', 'a', 'tactical', 'role', '@', '@', 'playing', 'video', 'game', 'developed', 'by', 'sega', 'and', 'mediavision', 'for', 'the', 'playstation', 'portable', 'released', 'in', 'january', '2011', 'in', 'japan', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'valkyria', 'series', 'employing', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', '@', '@', 'time', 'gameplay', 'as', 'its', 'predecessors', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the', '``', 'nameless', '``', 'a', 'penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 'gallia', 'during', 'the', 'second', 'europan', 'war', 'who', 'perform', 'secret', 'black', 'operations', 'and',

In [15]:
len(tokenized_texts)

1165029

#### Step 4: Build Vocabulary - Numericalization

In [16]:
#making vocabs - numericalization
_PAD_TOKEN = '[PAD]'
_CLS_TOKEN = '[CLS]'
_SEP_TOKEN = '[SEP]'
_MASK_TOKEN = '[MASK]'
word_list = list(set(" ".join([sent for sublist in tokenized_texts for sent in sublist]).split()))
word2id   = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}


In [17]:
# Build word2id
for i, w in enumerate(word_list):
    word2id[w] = i + 4  # reserve the first 0-3 for CLS, PAD

# Create id2word and vocab_size after word2id is complete
id2word = {i: w for w, i in word2id.items()}
vocab_size = len(word2id)

# Numericalize tokenized_texts
token_list = [
    [word2id[word] for word in sentence if word in word2id]
    for sentence in tokenized_texts
]

In [18]:
len(tokenized_texts)

1165029

In [19]:
for i, token in enumerate(token_list[:5]):
    print(f"Sample {i+1} token IDs: {token[:10]}...")  # Print first 10 token IDs for brevity

Sample 1 token IDs: [290398, 86317, 55900, 59127, 290398]...
Sample 2 token IDs: [389713, 385399, 86317, 17796, 467181, 415568, 55900, 171827, 418081, 467181]...
Sample 3 token IDs: [252381, 204127, 172171, 505155, 371455, 213713, 242738, 160963, 65705, 412394]...
Sample 4 token IDs: [150193, 491742, 33825, 141503, 164720, 371455, 386039, 83500, 338935, 353199]...
Sample 5 token IDs: [290398, 290398, 502282, 290398, 290398]...



### 2) Train the model on a suitable dataset. Ensure to source this dataset from reputable public databases or repositories. It is imperative to give proper credit to the dataset source in your documentation.


#### Step 5: Data Loader

We gonna make dataloader.  Inside here, we need to make two types of embeddings: **token embedding** and **segment embedding**

1. **Token embedding** - Given “The cat is walking. The dog is barking”, we add [CLS] and [SEP] >> “[CLS] the cat is walking [SEP] the dog is barking”. 

2. **Segment embedding**
A segment embedding separates two sentences, i.e., [0 0 0 0 1 1 1 1 ]

3. **Masking**
As mentioned in the original paper, BERT randomly assigns masks to 15% of the sequence. In this 15%, 80% is replaced with masks, while 10% is replaced with random tokens, and the rest 10% is left as is.  Here we specified `max_pred` 

4. **Padding**
Once we mask, we will add padding. For simplicity, here we padded until some specified `max_len`. 

Note:  `positive` and `negative` are just simply counts to keep track of the batch size.  `positive` refers to two sentences that are really next to one another.

In [20]:
batch_size = 6
max_mask   = 5 #even though it does not reach 15% yet....maybe you can set this threshold
max_len    = 1000 #maximum length that my transformer will accept.....all sentence will be padded

In [21]:
from random import randint, randrange
from random import shuffle
from random import random



def make_batch():
    batch = []
    positive = negative = 0
    while positive != batch_size / 2 or negative != batch_size / 2:
        
        #randomly choose two sentence
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
        tokens_a, tokens_b            = token_list[tokens_a_index], token_list[tokens_b_index]
        
        #1. token embedding - add CLS and SEP
        input_ids = [word2id['[CLS]']] + tokens_a + [word2id['[SEP]']] + tokens_b + [word2id['[SEP]']]
        
        #2. segment embedding - which sentence is 0 and 1
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)
        
        #3 masking
        n_pred = min(max_mask, max(1, int(round(len(input_ids) * 0.15))))
        #get all the pos excluding CLS and SEP
        candidates_masked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]'] 
                                 and token != word2id['[SEP]']]
        shuffle(candidates_masked_pos)
        masked_tokens, masked_pos = [], []
        #simply loop and mask accordingly
        for pos in candidates_masked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            if random() < 0.1:  #10% replace with random token
                index = randint(0, vocab_size - 1)
                input_ids[pos] = word2id[id2word[index]]
            elif random() < 0.8:  #80 replace with [MASK]
                input_ids[pos] = word2id['[MASK]']
            else: 
                pass
            
        #4. pad the sentence to the max length
        n_pad = max_len - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)
        
        #5. pad the mask tokens to the max length
        if max_mask > n_pred:
            n_pad = max_mask - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)
        
        #6. check whether is positive or negative
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True])
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False])
            negative += 1
        
    return batch
        

In [22]:
batch = make_batch()

In [23]:
len(batch)

6

In [24]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

In [25]:
input_ids.shape, segment_ids.shape, masked_tokens.shape, masked_pos.shape, isNext

(torch.Size([6, 1000]),
 torch.Size([6, 1000]),
 torch.Size([6, 5]),
 torch.Size([6, 5]),
 tensor([0, 0, 0, 1, 1, 1]))

In [26]:
masked_tokens

tensor([[159136,  83500, 353630, 170062, 252381],
        [171827, 445011, 353630, 317874, 172171],
        [290398, 143500, 290398,      0,      0],
        [311278, 457581,  80590, 345657,  80539],
        [510825,  14529, 510825, 352329, 388523],
        [ 83500, 296795, 252381, 344607, 325249]])

#### Step 6. Model



Recall that BERT only uses the encoder.

BERT has the following components:

- Embedding layers
- Attention Mask
- Encoder layer
- Multi-head attention
- Scaled dot product attention
- Position-wise feed-forward network
- BERT (assembling all the components)

##### 6.1 Embedding

In [27]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, max_len, n_segments, d_model, device):
        super(Embedding, self).__init__()
        self.device = device
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_len, d_model)      # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        #x, seg: (bs, len)
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long, device=x.device)
        pos = pos.unsqueeze(0).expand_as(x)  # (len,) -> (bs, len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

##### 6.2 Attention mask

In [28]:
def get_attn_pad_mask(seq_q, seq_k, device=None):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

In [29]:
print(get_attn_pad_mask(input_ids, input_ids).shape)

torch.Size([6, 1000, 1000])


##### 6.3 Encoder

The encoder has two main components: 

- Multi-head Attention
- Position-wise feed-forward network

First let's make the wrapper called `EncoderLayer`

In [30]:
class EncoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, d_ff, d_k, device):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(n_heads, d_model, d_k)
        self.pos_ffn       = PoswiseFeedForwardNet(d_model, d_ff)

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

Let's define the scaled dot attention, to be used inside the multihead attention

In [31]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super(ScaledDotProductAttention, self).__init__()
        self.d_k = d_k

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn 

Let's define the parameters first

In [32]:
n_layers = 6    # number of Encoder of Encoder Layer
n_heads  = 8    # number of heads in Multi-Head Attention
d_model  = 768  # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

Here is the Multiheadattention.

In [39]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_model, d_k):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = n_heads
        self.d_k = d_k
        self.d_v = d_k  # d_v = d_k
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_k * n_heads)
        self.linear = nn.Linear(n_heads * d_k, d_model)
        self.norm = nn.LayerNorm(d_model)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)
        k_s = self.W_K(K).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)
        v_s = self.W_V(V).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)

        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1)

        context, attn = ScaledDotProductAttention(self.d_k)(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v)
        output = self.linear(context)
        return self.norm(output + residual), attn

Here is the PoswiseFeedForwardNet.

In [40]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(F.gelu(self.fc1(x)))


##### 6.4 Putting them together

In [41]:
class BERT(nn.Module):
    def __init__(self, n_layers, n_heads, d_model, d_ff, d_k, n_segments, vocab_size, max_len, device):
        super(BERT, self).__init__()
        self.params = {'n_layers': n_layers, 'n_heads': n_heads, 'd_model': d_model,
                       'd_ff': d_ff, 'd_k': d_k, 'n_segments': n_segments,
                       'vocab_size': vocab_size, 'max_len': max_len}
        self.embedding = Embedding(vocab_size, max_len, n_segments, d_model, device)
        self.layers = nn.ModuleList([EncoderLayer(n_heads, d_model, d_ff, d_k, device) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))
        self.device = device

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids, self.device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
        
        # 1. predict next sentence
        # it will be decided by first token(CLS)
        h_pooled   = self.activ(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_nsp = self.classifier(h_pooled) # [batch_size, 2]

        # 2. predict the masked token
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked  = self.norm(F.gelu(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_nsp
    
    def get_last_hidden_state(self, input_ids, segment_ids):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids, self.device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)

        return output

#### Step 7. Training

In [42]:
import numpy as np
import torch.nn.functional as F

num_epoch = 20 #500
model = BERT(
    n_layers, 
    n_heads, 
    d_model, 
    d_ff, 
    d_k, 
    n_segments, 
    vocab_size, 
    max_len, 
    device
).to(device) 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

# Move tensors to the same device as the model
input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
masked_tokens = masked_tokens.to(device)
masked_pos = masked_pos.to(device)
isNext = isNext.to(device)

for epoch in range(num_epoch):
    optimizer.zero_grad()
    logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos) 
    #logits_lm: (bs, max_mask, vocab_size) ==> (6, 5, 34)
    #logits_nsp: (bs, yes/no) ==> (6, 2)

    #1. mlm loss
    #logits_lm.transpose: (bs, vocab_size, max_mask) vs. masked_tokens: (bs, max_mask)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    #2. nsp loss
    #logits_nsp: (bs, 2) vs. isNext: (bs, )
    loss_nsp = criterion(logits_nsp, isNext) # for sentence classification
    
    #3. combine loss
    loss = loss_lm + loss_nsp
    if epoch % 5 == 0:
        print('Epoch:', '%02d' % (epoch), 'loss =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch: 00 loss = 118.843636
Epoch: 05 loss = 138.800888
Epoch: 10 loss = 57.787827
Epoch: 15 loss = 34.719833


#### Step 7. Inference

In [44]:
# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[2]))
input_ids, segment_ids, masked_tokens, masked_pos = input_ids.to(device), segment_ids.to(device), masked_tokens.to(device), masked_pos.to(device)
print([id2word[w.item()] for w in input_ids[0] if id2word[w.item()] != '[PAD]'])

logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)
#logits_lm:  (1, max_mask, vocab_size) ==> (1, 5, 34)
#logits_nsp: (1, yes/no) ==> (1, 2)

#predict masked tokens
#max the probability along the vocab dim (2), [1] is the indices of the max, and [0] is the first value
logits_lm = logits_lm.data.max(2)[1][0].data.cpu().numpy()
#note that zero is padding we add to the masked_tokens
print('masked tokens (words) : ',[id2word[pos.item()] for pos in masked_tokens[0]])
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0]])
print('masked tokens (words) : ',[id2word[pos.item()] for pos in logits_lm])
print('predict masked tokens list : ', [pos for pos in logits_lm])

#predict nsp
logits_nsp = logits_nsp.data.max(1)[1][0].data.cpu().numpy()
print(logits_nsp)
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_nsp else False)

['[CLS]', '``', 'turn', 'up', 'the', 'music', '``', 'also', 'reached', 'the', 'top', 'twenty', 'in', 'some', 'european', 'countries', 'it', 'peaked', 'at', 'number', '15', 'on', 'the', 'hungarian', 'airplay', 'chart', 'and', 'number', '20', 'on', 'the', 'slovak', 'airplay', 'chart', 'on', 'the', 'irish', 'singles', 'chart', '``', 'turn', 'up', 'the', 'music', '``', 'peaked', 'at', 'number', '12', 'and', 'spent', '18', 'weeks', 'in', 'the', 'chart', 'in', 'the', 'united', 'kingdom', 'the', 'song', 'first', 'charted', 'on', '[MASK]', '10', '2012', 'in', 'a', 'cover', 'version', 'by', 'beautiful', 'people', 'reaching', 'the', 'top', '100', 'on', 'april', '7', '2012', 'the', 'chris', 'brown', 'version', 'having', 'sold', '83', '@', '@', '777', 'copies', 'in', 'its', 'first', 'week', 'became', 'brown', "'s", 'first', 'uk', 'number', 'one', 'single', 'that', 'same', 'week', 'it', 'also', 'debuted', 'at', 'number', 'one', 'on', 'the', 'scottish', 'singles', 'chart', '``', 'turn', 'up', 'the',


### 3) Save the trained model weights for later use in Task 2.
NOTE: BERT-update.ipynb is available to use CUDA.
NOTE: You may refer to the BERT $paper^1$ and use large corpora such as $BookCorpus^2$ or English
$Wikipedia^3$. However, you should only use a subset, such as 100k samples, rather than the entire dataset.

1 https://aclanthology.org/N19-1423.pdf

2 https://huggingface.co/datasets/bookcorpus/bookcorpus

3 https://huggingface.co/datasets/legacy-datasets/wikipedia


In [45]:
# Save checkpoints
_MODEL_CHECKPOINT_PATH = "../models"
_MODEL_CHECKPOINT_FILENAME = "bert_checkpoint.pt"
checkpoint_full_path = _MODEL_CHECKPOINT_PATH+ "/" + _MODEL_CHECKPOINT_FILENAME

torch.save([model.params, model.state_dict()], checkpoint_full_path)
print("BERT Model checkpoint saved!")

BERT Model checkpoint saved!


In [46]:
# Load checkpoint
params, state_dict = torch.load(f'{checkpoint_full_path}', map_location=device)

# Recreate model with saved hyperparameters
loaded_model = BERT(**params, device=device).to(device)

# Load weights
loaded_model.load_state_dict(state_dict)
loaded_model.eval()

print("Model loaded from checkpoint!")

Model loaded from checkpoint!


## Task 2. Sentence Embedding with Sentence BERT - Implement trained BERT from task 1 with
siamese network structures to derive semantically meaningful sentence embeddings that can be compared
using cosine-similarity. (3 points)
1) Use the $SNLI^4$ OR $MNLI^5$ datasets from Hugging Face, or any dataset related to classification
tasks.
2) Reproduce training the Sentence-BERT as described in the $paper^6$.
3) Focus on the Classification Objective Function: (SoftmaxLoss)
$$o = softmax (W^T·(u, v, |u −v|))$$
HINT : You can take a look how to implement Softmax loss in the file 04 - Huggingface/Appendix -
Sentence Embedding/S-BERT.ipynb.
1https://aclanthology.org/N19-1423.pdf2https://huggingface.co/datasets/bookcorpus/bookcorpus3https://huggingface.co/datasets/legacy-datasets/wikipedia4https://huggingface.co/datasets/snli5https://huggingface.co/datasets/glue/viewer/mnli6https://aclanthology.org/D19-1410/
1
2

4 https://huggingface.co/datasets/stanfordnlp/snli

5 https://huggingface.co/datasets/glue/viewer/mnli

6 https://aclanthology.org/D19-1410/


## Task 3. Evaluation and Analysis (1 points)
1) Provide the performance metrics (classification Report) based on the SNLI or MNLI datasets for
the Natural Language Inference (NLI) task.
precision recall f1-score support
entailment 0.42 0.02 0.05 3486
neutral 0.33 0.75 0.46 3199
contradiction 0.33 0.25 0.28 3315
accuracy 0.33 10000
macro avg 0.36 0.34 0.26 10000
weighted avg 0.36 0.33 0.26 10000
Table 1. Sample of Classification Report
2) Discuss any limitations or challenges encountered during the implementation and propose potential
improvements or modifications.
NOTE: Make sure to provide proper documentation, including details of the datasets used, hyperpa-
rameters, and any modifications made to the original models.


## Task 4. Text similarity - Web Application Development - Develop a simple web application that
demonstrates the capabilities of your text-embedding model. (1 points)
1) Develop a simple website with two input boxes for search queries.
2) Utilize a custom-trained sentence transformer model to predict Natural Language Inference (NLI)
Task (entailment, neutral and contradiction).
For example:
• Premise: A man is playing a guitar on stage.
• Hypothesis: The man is performing music.
• Label: Entailment